In [1]:
from dolfin import *
import mshr
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML 
import time

In [2]:
def analytic2(mu, rho, T, L, dt, cnt):   
    
    a = []
    omega = []
    for i in range(1,4):
        a.append(4000/i)
        omega.append(i * 2 * np.pi)

    x = np.linspace(0, L, 40)
    
    def cc(x):
        return np.cos(x)*np.cosh(x)

    def ss(x):
        return np.sin(x)*np.sinh(x)

    def periodic(t, omega, a):
        k = np.sqrt(omega*rho/2./mu)
        xs = x - L/2
        f1 = cc(k*xs)*cc(k*L/2) + ss(k*xs)*ss(k*L/2)
        f2 = cc(k*xs)*ss(k*L/2) - ss(k*xs)*cc(k*L/2)
        f3 = cc(k*L/2)**2 + ss(k*L/2)**2
        u = a/omega*(f2/f3*np.sin(omega*t) - (1 - f1/f3)*np.cos(omega*t))
        return u
    def transient(k, t, omega, a):
        d = 4./(rho*np.pi*(2*k + 1)*((2*k + 1)**4*(mu/rho/L**2)**2*np.pi**4 +
                                 omega**2))
        S = np.sin(np.pi*(2*k + 1)*x/L)
        psi = ((2*k + 1)**2*mu/rho/L**2*np.pi**2*np.sin(omega*t) -
               omega*np.cos(omega*t) +
               omega*np.exp(-(2*k + 1)**2*mu/rho/L**2*np.pi**2*t))

        return a*d*S*psi
    t = np.arange(0, T, dt)
    #plt.ylim(-1.2, 2)
    #plt.xlim(0, L)
    up = np.zeros(shape=(len(t),len(x)))
    ut = np.zeros(shape=(len(t),len(x)))
              
    count = 0

    for t_ in t:      
        up_var = periodic(t_, omega[cnt], a[cnt])
        up[count] += up_var
        ut_var = 0
        for k in range(10):
            ut_var += transient(k, t_, omega[cnt], a[cnt])
        ut[count] += ut_var
        count += 1
    mmax = []
    mmin = []
    for i in range(len(ut)):
        mmax.append(max(max(ut[i]), max(up[i])))
        mmin.append(min(min(ut[i]), min(up[i])))
    
    
    mmax = 1.1*max(mmax)
    mmin = 1.1*min(mmin)
    fig, ax = plt.subplots()
    ax.axis([0, L,mmin, mmax])
    pper,=ax.plot([],[])
    ptrans,=ax.plot([],[])
    
    def ani(i): 
        #pper.set_data(x, up[i])
        
        ptrans.set_data(x, ut[i])
    ani = matplotlib.animation.FuncAnimation(fig, ani, frames=len(t))
    
    return ani



In [22]:
def analytic(mu, rho, T, L, dt, u_line, progress_flag, proceents):   
    
    a = []
    omega = []
    for i in range(1,4):
        a.append(4000/i)
        omega.append(i * 2 * np.pi)

    x = np.linspace(0, L, 40)
    
    def cc(x):
        return np.cos(x)*np.cosh(x)

    def ss(x):
        return np.sin(x)*np.sinh(x)

    def periodic(t, omega, a):
        k = np.sqrt(omega*rho/2./mu)
        xs = x - L/2
        f1 = cc(k*xs)*cc(k*L/2) + ss(k*xs)*ss(k*L/2)
        f2 = cc(k*xs)*ss(k*L/2) - ss(k*xs)*cc(k*L/2)
        f3 = cc(k*L/2)**2 + ss(k*L/2)**2
        u = a/omega*(f2/f3*np.sin(omega*t) - (1 - f1/f3)*np.cos(omega*t))
        return u
    def transient(k, t, omega, a):
        d = 4./(rho*np.pi*(2*k + 1)*((2*k + 1)**4*(mu/rho/L**2)**2*np.pi**4 +
                                 omega**2))
        S = np.sin(np.pi*(2*k + 1)*x/L)
        psi = ((2*k + 1)**2*mu/rho/L**2*np.pi**2*np.sin(omega*t) -
               omega*np.cos(omega*t) +
               omega*np.exp(-(2*k + 1)**2*mu/rho/L**2*np.pi**2*t))

        return a*d*S*psi
    razn = 3*dt
    t = np.arange(0 + razn, T + razn, dt)
    #plt.ylim(-1.2, 2)
    #plt.xlim(0, L)
    up = np.zeros(shape=(len(t),len(x)))
    ut = np.zeros(shape=(len(t),len(x)))
              
    count = 0
    proceent = 0
    for t_ in t:
        if progress_flag == 1:
            if int(t_/(T - dt)*100) >= proceent:
                print("O/o =", int(t_/(T - dt)* 100))
                proceent+=proceents
        for i in range(3):
            up_var = periodic(t_, omega[i], a[i])
            up[count] += up_var
            ut_var = 0
            for k in range(10):
                ut_var += transient(k, t_, omega[i], a[i])
            ut[count] += ut_var
        count += 1
    ut = ut
    up = up
    mmax = []
    mmin = []
    for i in range(len(u_line)):
        mmax.append(max(max(u_line[i]), max(up[i]), max(ut[i])))
        mmin.append(min(min(u_line[i]), min(up[i]), min(ut[i])))
    
    
    mmax = 1.1*max(mmax)
    mmin = 1.1*min(mmin)
    fig, ax = plt.subplots()
    ax.axis([0, L,mmin, mmax])
    pper,=ax.plot([],[])
    ptrans,=ax.plot([],[])
    uline,=ax.plot([],[])
    
    def ani(i): 
        #pper.set_data(x, up[i])
        
        ptrans.set_data(x, ut[i])
        uline.set_data(x, u_line[i])
    ani = matplotlib.animation.FuncAnimation(fig, ani, frames=len(t))
    
    return ani


In [4]:
def stokes_problem_no_p(N, Len, T, num_steps, mu, rho, epsilon, theta, p_flag, plot_flag, element_n, progress_flag, proceents, t_start):     
    dt = T / num_steps # time step size
    t = dt
    p_f = p_flag
    
    mesh = RectangleMesh(Point(0, -1), Point(Len, 1), 2*N, Len*N)
    h = CellDiameter(mesh)
    
    # Build function space
    W = FunctionSpace(mesh, "P", element_n)
    walls   = 'near(x[1], -1) || near(x[1], 1)'

    # Define boundary conditions
    bcu_noslip  = DirichletBC(W, Constant(0), walls)
    bcs = [bcu_noslip]

    # Define variational problem
    u = TrialFunction(W)
    v = TestFunction(W)
    u0 = project(Constant(0), W)
    vc = 4000/1*np.sin(2 * np.pi * t) + 4000/2*np.sin(4 * np.pi * t) + 4000/3*np.sin(6 * np.pi * t)
    f = Constant(vc)
    vc0 = 4000/1*np.sin(2 * np.pi * (t-dt)) + 4000/2*np.sin(4 * np.pi * (t-dt)) + 4000/3*np.sin(6 * np.pi * (t-dt))
    f0 = Constant(vc0)

    #folder = 'N' + str(N) + 'L' + str(L) + 'dt' + str(dt) + 'mu' + str(mu) + 'rho' + str(rho) + 'epsilon' + str(epsilon) + 'theta' + str(theta) + 'p_flag' + str(p_flag)
    folder = 'pf=' + str(p_f) + ',dt=' + str(dt) + ',e=' + str(epsilon) + ',th=' + str(theta)  
    
    u_file = XDMFFile(folder + '/u.xdmf')
    u_file.parameters['rewrite_function_mesh'] = False
    u_file.parameters["flush_output"] = True

    # Time-stepping
    proceent = 0
    
    vc_arr = np.zeros(num_steps)
    t_arr = np.zeros(num_steps)
    u_line_arr = []
    
    
    for n in range(num_steps):
        if progress_flag == 1:
            if int(n/(num_steps-1)*100) >= proceent:
                print("% =", int(n/(num_steps-1) * 100), ', time mplsd =', int((time.time()-t_start)))
                proceent+=proceents
                #c = plot(u0)
                #plt.colorbar(c)
                #plt.show()
        vc = 4000/1*np.sin(2 * np.pi * t) + 4000/2*np.sin(4 * np.pi * t) + 4000/3*np.sin(6 * np.pi * t)
        f = Constant(-vc)
        vc0 = 4000/1*np.sin(2 * np.pi * (t-dt)) + 4000/2*np.sin(4 * np.pi * (t-dt)) + 4000/3*np.sin(6 * np.pi * (t-dt))
        f0 = Constant(-vc0)
        a1 = (mu*dot(grad(u0), grad(v)))*dx
        L1 = f0*v/Len*dx
        a2 = (mu*dot(grad(u), grad(v)))*dx
        L2 = f*v/Len*dx
        
        vc_arr[n] = vc
        t_arr[n] = t
        U = Function(W)
        a = dot(u,v) * dx + (theta) * (a2)
        L =  (1 - theta) * (a1 - L1) - (theta) * L2 - dot(u0,v)*dx
        solve(a == L, U, bcs)
        u0 = U
        u0.rename("v0", "velocity")
        u_file.write(u0, t)
        t += dt
        x_ = 3
        y = np.linspace(-1, 1, 40)
        u_line = []
        #print(type(L))
        for i in y:
            u_line.append(u0(x_, i))
        u_line_arr.append(u_line)


    #file close
    u_file.close()
    if plot_flag == 1:
        plt.plot(t_arr,vc_arr)
    return u_line_arr

In [17]:
def stokes_problem_p(N, Len, T, num_steps, mu, rho, epsilon, theta, p_flag, plot_flag, element_n, progress_flag, proceents, t_start):     
    dt = T / num_steps # time step size
    t = dt
    p_f = p_flag
    
    mesh = RectangleMesh(Point(0, -1), Point(Len, 1), 2*N, Len*N)
    h = CellDiameter(mesh)
    
    # Build function space
    P2 = VectorElement("P", mesh.ufl_cell(), element_n)
    P1 = FiniteElement("P", mesh.ufl_cell(), 1)
    TH = P2 * P1
    W = FunctionSpace(mesh, TH)
    walls   = 'near(x[1], -1) || near(x[1], 1)'

    # Define boundary conditions
    bcu_noslip  = DirichletBC(W.sub(0), Constant((0, 0)), walls)
    bcs = [bcu_noslip]

    # Define variational problem
    (u, p) = TrialFunctions(W)
    (v, q) = TestFunctions(W)
    u0 = project(Constant((0, 0)), VectorFunctionSpace(mesh, "P", 4))
    p0 = project(Constant(0), FunctionSpace(mesh, "P", 4))



    vc = 4000/1*np.sin(2 * np.pi * t) + 4000/2*np.sin(4 * np.pi * t) + 4000/3*np.sin(6 * np.pi * t)
    f = Constant((vc, 0))
    vc0 = 4000/1*np.sin(2 * np.pi * (t-dt)) + 4000/2*np.sin(4 * np.pi * (t-dt)) + 4000/3*np.sin(6 * np.pi * (t-dt))
    f0 = Constant((vc0, 0))

    #folder = 'N' + str(N) + 'L' + str(L) + 'dt' + str(dt) + 'mu' + str(mu) + 'rho' + str(rho) + 'epsilon' + str(epsilon) + 'theta' + str(theta) + 'p_flag' + str(p_flag)
    folder = 'pf=' + str(p_f) + ',dt=' + str(dt) + ',e=' + str(epsilon) + ',th=' + str(theta)  
    
    u_file = XDMFFile(folder + '/u.xdmf')
    p_file = XDMFFile(folder + '/p.xdmf')
    u_file.parameters['rewrite_function_mesh'] = False
    p_file.parameters['rewrite_function_mesh'] = False
    u_file.parameters["flush_output"] = True
    p_file.parameters["flush_output"] = True

    # Time-stepping
    proceent = 0
    
    vc_arr = np.zeros(num_steps)
    t_arr = np.zeros(num_steps)
    u_line_arr = []
    
    for n in range(num_steps):
        if progress_flag == 1:
            if int(n/(num_steps-1)*100) >= proceent:
                print("% =", int(n/(num_steps-1) * 100), ', time mplsd =', int((time.time()-t_start)))
                proceent+=proceents
        vc = 4000/1*np.sin(2 * np.pi * t) + 4000/2*np.sin(4 * np.pi * t) + 4000/3*np.sin(6 * np.pi * t)
        f = Constant((vc, 0))
        vc0 = 4000/1*np.sin(2 * np.pi * (t-dt)) + 4000/2*np.sin(4 * np.pi * (t-dt)) + 4000/3*np.sin(6 * np.pi * (t-dt))
        f0 = Constant((vc0, 0))
        a1 = (mu*inner(grad(u0), grad(v)) + div(v)*p0 + mu*q*div(u0) + epsilon*h*h/mu*inner(grad(q), grad(p0)))*dx
        L1 = inner(f0, v)*dx
        a2 = (mu*inner(grad(u), grad(v)) + div(v)*p + mu*q*div(u) + epsilon*h*h/mu*inner(grad(q), grad(p)))*dx
        L2 = inner(f, v)*dx
    
        U = Function(W)
        
        vc_arr[n] = vc
        t_arr[n] = t
        
        a = inner(u,v) * dx + (theta) * (a2)
        L =  (1 - theta) * (a1 - L1) - (theta) * L2 - inner(u0,v)*dx
        solve(a == L, U, bcs)
        u0, p0 = U.split()
        u0.rename("v0", "velocity")
        p0.rename("p0", "pressure")
        u_file.write(u0, t)
        p_file.write(p0, t)
        t += dt
        x_ = 3
        y = np.linspace(-1, 1, 40)
        u_line = []
        for i in y:
            u_line.append(u0(x_, i)[0])
        u_line_arr.append(u_line)
    #file close
    u_file.close()
    p_file.close()
    if plot_flag == 1:
        plt.plot(t_arr,vc_arr)
    return u_line_arr

In [6]:
def stokes_problem(N, L, T, num_steps, mu, rho, epsilon, theta, p_flag, plot_flag, element_n, progress_flag, proceents, t_start):     
    if p_flag == 1:
        u_line_arr = stokes_problem_p(N, L, T, num_steps, mu, rho, epsilon, theta, p_flag, plot_flag, element_n, progress_flag, proceents, t_start)
    if p_flag == 0:
        u_line_arr = stokes_problem_no_p(N, L, T, num_steps, mu, rho, epsilon, theta, p_flag, plot_flag, element_n, progress_flag, proceents, t_start)
    return u_line_arr

assignment 1.2.5

In [1]:
t_start = time.time()
N = 16
L = 6
T = 2.           # final time
num_steps = 500    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0
theta = 1
p_flag = 0
f_plot_flag = 1
element_n = 1
progress_flag = 1
proceents = 10
u_line = stokes_problem(N, L, T, num_steps, mu, rho, epsilon, theta, p_flag, f_plot_flag, element_n, progress_flag, proceents, t_start)
u_line = np.array(u_line)

ani = analytic(mu, rho, T, 2, dt, u_line, progress_flag, proceents)
print('time mplsd =', int((time.time()-t_start)))
HTML(ani.to_jshtml())

NameError: name 'time' is not defined

In [2]:
t_start = time.time()
N = 16
L = 6
T = 2.           # final time
num_steps = 1000    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0
theta = 0.5
p_flag = 0
f_plot_flag = 1
element_n = 1
progress_flag = 1
proceents = 10
u_line = stokes_problem(N, L, T, num_steps, mu, rho, epsilon, theta, p_flag, f_plot_flag, element_n, progress_flag, proceents, t_start)
u_line = np.array(u_line)

ani = analytic(mu, rho, T, 2, dt, u_line, progress_flag, proceents)
print('time mplsd =', int((time.time()-t_start)))
HTML(ani.to_jshtml())

NameError: name 'time' is not defined

assignment 1.2.6

In [3]:
print('time mplsd =', int((time.time()-t_start)))
t_start = time.time()
N = 4
L = 6
T = 2           # final time
num_steps = 20000    # number of time steps
dt = T/num_steps
theta = 0
u_line = stokes_problem(N, L, T, num_steps, mu, rho, epsilon, theta, p_flag, f_plot_flag, element_n, progress_flag, proceents, t_start)
u_line = np.array(u_line)
ani = analytic(mu, rho, T, L, dt, u_line, progress_flag, proceents)
print('time mplsd =', int((time.time()-t_start)))
HTML(ani.to_jshtml())

NameError: name 'time' is not defined

assignment 1.2.7

The CFL condition then has the following form:
$$
C=\frac{u \Delta t}{h} \leq C_{\max }
$$
so with decreasing the $h$ or increasin of the $N_y$ we overcome the $C_{\max }$ and the solution becomes unstable

assignment 1.3.3

$$
\begin{aligned}
\rho \int_{\Omega}{\frac{\vec{u}_{n}^{k+1} -\vec{u}_{n}^{k+n}}{\tau}} \cdot \vec{v}_{n}+\mu \int_{\Omega} \nabla \vec{u}_{n}^{k+\theta}: \nabla \vec{v}_{n} \\
&-\int_{\Omega}{p^{k+\theta} \nabla \cdot \vec{v}_{n}}+\int_{\Omega} g_{n} \nabla \cdot \vec{u}_{n}^{k+\theta}=0
\end{aligned}
$$

$
\text { Lyapunov: } \quad \vec{v}_{n}=\vec{u}_{n}^{k+\theta}
$

Energy balance for discrete Stokes
$$
\begin{aligned}
\frac{\rho}{2} \int_{\Omega}\left\|\vec{u}_{n}^{k+1}\right\|^{2}-\frac{\rho}{2} &\int_{\Omega}{ \| \vec{u}_{n}^{k} \|^{2}}+\left(\theta-\frac{1}{2}\right)\int_{\Omega}{\left\|u_{n}^{k+1}-\vec{u}_{n}^{k}\right\|^{2}} \\
&=-\tau \mu \int_{\Omega}\left\|\nabla \vec{u}_{n}^{k+\theta}\right\|^{2}
\end{aligned}
$$

Note: pressure does not appear because $0=\int_{\Omega} g_{n} \nabla \cdot u_{n}^{k+1}$, where $g_{n} = p_{n}^{k+\theta}$ 

Unconditionally stable for $\theta \geqslant 1 / 2 $

assignment 1.4.2 

for epsilon stabilization we have an additional term 
$$
\frac{\varepsilon h^{2}}{\mu} \int_{\Omega} \nabla p_{h}^{k+\theta} \cdot \nabla g_{n}=0
$$

Stabilized stokes $\left(\mathbb{P}_{1}\left(\mathbb{P}_{1}\right)\right.$
$\rho \int_{\Omega} \frac{\vec{u}_{n}^{k+1}-\vec u_{n}^{k}}{\tau} \cdot \vec{v}_{n}+\mu \int_{\Omega} \nabla \vec{u}_{n}^{k+\theta}: \nabla \vec{v}_{n}$

$-\int_{\Omega} P_{h}^{k+\theta} \nabla \cdot \vec{v}_{n}+\int_{\Omega} g_{n} \nabla \cdot \vec{u}_{n}^{k+\theta}$

$\vec{v}_{n}=\vec{u}_{n}^{k+\theta}$, $g_{n}=p_{h}^{k+\theta}$

$+\frac{\varepsilon h^{2}}{\mu} \int_{\Omega} \nabla_{p_{h}^{k+\theta}} \cdot \nabla g_{n}=0$


$E_{K+1}-E_{k}=\tau D_{K+\theta}+\left(\frac{1}{2}-\theta\right) \int_{\Omega} \rho\left\|\cdot u_{n}^{k+1}-u_{n}^{k}\right\|^{2}$

$-\int_{\Omega} p_{n}^{k+\theta} \nabla \cdot u_{n}^{k+\theta}+\int_{\Omega} p_{n}^{k+\theta} \nabla \cdot \vec{u}_{n}^{k+\theta}-\frac{\tau \varepsilon h^{2}}{\mu} \int_{\Omega} \| \nabla p_{n}^{k+\theta} \|^{2}$



$E_{K+1}-E_{k}=\tau D_{K+\theta}+\left(\frac{1}{2}-\theta\right) \int_{\Omega} \rho\left\|\cdot u_{n}^{k+1}-u_{n}^{k}\right\|^{2}-\frac{\tau \varepsilon h^{2}}{\mu} \int_{\Omega} \| \nabla p_{n}^{k+\theta} \|^{2}$

if theta is more or equals than 1/2

$E_{k+1}< E_{k}$ since $\tau D_{K+\theta} < 0$ and $-\frac{\tau \varepsilon h^{2}}{\mu} \int_{\Omega} \| \nabla p_{n}^{k+\theta} \|^{2}<0$

assignment 1.3.4

In [4]:
N = 16
L = 6
T = 10           # final time
num_steps = 1000    # number of time steps
dt = T/num_steps
mu = 0.035             # kinematic viscosity
rho = 1.2            # density
epsilon = 0
theta = 0.5
p_flag = 1
f_plot_flag = 0
element_n = 2
progress_flag = 0
t_start = time.time()
u_line = stokes_problem(N, L, T, num_steps, mu, rho, epsilon, theta, p_flag, f_plot_flag, element_n, progress_flag, proceents, t_start)
u_line = np.array(u_line)
ani = analytic(mu, rho, T, 2, dt, u_line, progress_flag, proceents)
print('time mplsd =', int((time.time()-t_start)))
HTML(ani.to_jshtml())

NameError: name 'time' is not defined

assignment 1.3.5

In [5]:
N = 32
L = 6
T = 2           # final time
num_steps = 100    # number of time steps
dt = T/num_steps
mu = 0.035             # kinematic viscosity
rho = 1.2            # density
epsilon = 0
theta = 0.1
p_flag = 1
f_plot_flag = 0
element_n = 2
progress_flag = 1
t_start = time.time()
proceents = 10
u_line = stokes_problem(N, L, T, num_steps, mu, rho, epsilon, theta, p_flag, f_plot_flag, element_n, progress_flag, proceents, t_start)
u_line = np.array(u_line)
ani = analytic(mu, rho, T, 2, dt, u_line, progress_flag, proceents)
print('time mplsd =', int((time.time()-t_start)))
HTML(ani.to_jshtml())

NameError: name 'time' is not defined

In [6]:
N = 16
L = 6
T = 2           # final time
num_steps = 5000    # number of time steps
dt = T/num_steps
mu = 0.035             # kinematic viscosity
rho = 1.2            # density
epsilon = 0
theta = 0.1
p_flag = 1
f_plot_flag = 0
element_n = 2
progress_flag = 1
t_start = time.time()
u_line = stokes_problem(N, L, T, num_steps, mu, rho, epsilon, theta, p_flag, f_plot_flag, element_n, progress_flag, proceents, t_start)
u_line = np.array(u_line)
ani = analytic(mu, rho, T, 2, dt, u_line, progress_flag, proceents)
print('time mplsd =', int((time.time()-t_start)))
HTML(ani.to_jshtml())

NameError: name 'time' is not defined

assignment 1.4.2

assignment 1.4.3

In [7]:
N = 16
L = 6
T = 2           # final time
num_steps = 500    # number of time steps
dt = T/num_steps
mu = 0.035             # kinematic viscosity
rho = 1.2            # density
epsilon = 0.1
theta = 0.5
p_flag = 1
f_plot_flag = 0
element_n = 2
progress_flag = 1
t_start = time.time()
u_line = stokes_problem(N, L, T, num_steps, mu, rho, epsilon, theta, p_flag, f_plot_flag, element_n, progress_flag, proceents, t_start)
u_line = np.array(u_line)
ani = analytic(mu, rho, T, 2, dt, u_line, progress_flag, proceents)
print('time mplsd =', int((time.time()-t_start)))
HTML(ani.to_jshtml())

NameError: name 'time' is not defined

In [8]:
N = 16
L = 6
T = 2           # final time
num_steps = 1000    # number of time steps
dt = T/num_steps
mu = 0.035             # kinematic viscosity
rho = 1.2            # density
epsilon = 5
theta = 0.5
p_flag = 1
f_plot_flag = 0
element_n = 2
progress_flag = 0
t_start = time.time()
u_line = stokes_problem(N, L, T, num_steps, mu, rho, epsilon, theta, p_flag, f_plot_flag, element_n, progress_flag, proceents, t_start)
u_line = np.array(u_line)
ani = analytic(mu, rho, T, 2, dt, u_line, progress_flag, proceents)
print('time mplsd =', int((time.time()-t_start)))
HTML(ani.to_jshtml())

NameError: name 'time' is not defined

In [9]:
t_start = time.time()
N = 16
L = 6
T = 100           # final time
num_steps = 1000    # number of time steps
dt = T/num_steps
mu = 2            # kinematic viscosity
rho = 1.2            # density
epsilon = 1
theta = 0.5
p_flag = 0
f_plot_flag = 1
element_n = 1
progress_flag = 1
proceents = 10
u_line = stokes_problem(N, L, T, num_steps, mu, rho, epsilon, theta, p_flag, f_plot_flag, element_n, progress_flag, proceents, t_start)
u_line = np.array(u_line)

ani = analytic(mu, rho, T, 2, dt, u_line, progress_flag, proceents)
print('time mplsd =', int((time.time()-t_start)))
HTML(ani.to_jshtml())

NameError: name 'time' is not defined